### Hashing y LSH 2020-2C

Contamos con una colección de 1300 millones de tweets con fake news. Queremos usarlos
para detectar otros tweets que puedan contener mensajes parecidos para poder filtrarlos.

El objetivo final de nuestra aplicación es dado un tweet buscar cuáles son tweets parecidos en
nuestra base de 1300 millones de tweets usando LSH para luego decidir si el tweet es o no un
fake news. Si hay tweets muy similares al nuevo en nuestra base de datos, lo categorizamos
como fake news; caso contrario, no. El criterio que fijamos es que si el tweet actual tiene una
__semejanza mayor o igual a 0.73__ con alguno de nuestra base de fake news entonces es un fake
news.

Se decide usar la __distancia de Jaccard__ como métrica para la construcción de nuestro __esquema
de LSH__.

Queremos que si la __semejanza entre tweets es mayor o igual a 0.55__ tengamos __más de un 70%
de probabilidad de que sean candidatos__. Por otro lado, si la semejanza es __menor o igual a 0.05__
queremos tener __menos de un 1% de probabilidad de que sean candidatos__ a ser comparados.


En base a esta información le pedimos que responda las siguientes preguntas:

1. ¿Cuántos minhash hacen falta y con qué tipo de esquema (b y r) ? Detalle los cálculos
realizados y estime cantidad de falsos positivos y falsos negativos que vamos a tener
sobre una base de 10000 tweets nuevos. (30 puntos)

2. Describa la etapa de pre-procesamiento en la cual tiene que recorrer los 1300 millones
de tweets y crear una única tabla de hash. Recuerde que puede usar diagramas,
esquemas y pseudocódigo para hacer más clara su explicación. Debe ser claro
indicando cómo queda conformado el esquema LSH que va a usar en el punto 3. (35
puntos)

3. Describir cómo se hace la predicción de si un tweet es fake news o no utilizando el
esquema LSH propuesto. (35 puntos)

#### Ejercicio 1 

¿Cuántos minhash hacen falta y con qué tipo de esquema (b y r) ? Detalle los cálculos realizados y estime cantidad de falsos positivos y falsos negativos que vamos a tener sobre una base de 10000 tweets nuevos. (30 puntos)

----

Se parte de que si la semejanza entre dos tweets, es decir $\mathcal{J}(T_1, T_2) \geq 0.55 = d_1$ la probabilidad de colisión tiene que ser $\mathbb{P}("Colision") \geq 0.7 = p_1$, cuando $\mathcal{J}(T_1, T_2) \leq 0.05 = d_2$, queremos evitarlos con una probabilidad $\mathbb{P}("Colision") \leq 0.01 = p_2$.

Como $\mathbb{P}("Colision") = 1 - (1 - d^r)^b $

De la primer restricción se consigue:

$\mathbb{P}("Colision") = 1 - (1 - 0.55^r)^b \geq 0.7$

Y de la segunda:

$\mathbb{P}("Colision") = 1 - (1 - 0.05^r)^b \leq 0.01$

Para encontrar los valores que satisfacen estas condiciones se realizará $\verb|gridsearch|$

|  $r$ | $b$ |$p_1$|$p_2$|
|----|----|---|---|
|2|2|0.5134|0.0004|
|2|3|0.6606|0.0007|
|2|4|0.7633|0.01|

De lo que se concluye que $b = 4$ ($\verb|or|$) y $r = 2 $ ($\verb|and|$)


Los falsos positivos serán aquellos que fueron recuperados en la consulta pero que originalmente no eran deseados, es decir:

$\mathbb{P}("F^+") = 1 - p_1 = 1 - 0.7633 = 0.2367$

Por otro lado, los falsos negativos, son aquellos que se deberían haber recuperado en una consulta, pero que no lo hicieron (este tipo de error es más crítico):

$\mathbb{P}("F^-") = p_2 = 0.01$

Con lo cual estimando en el set completo de 10000:

$|F^+| \approx 10000 * 0.2367 = 2367$

$|F^-| \approx 10000 * 0.01 = 100$


#### Ejercicio 2

Describa la etapa de pre-procesamiento en la cual tiene que recorrer los 1300 millones de tweets y crear una única tabla de hash. Recuerde que puede usar diagramas, esquemas y pseudocódigo para hacer más clara su explicación. Debe ser claro indicando cómo queda conformado el esquema LSH que va a usar en el punto 3. (35 puntos)

----
Como pre-procesamiento de los datos, se tomará cada twit, se desompondrá en shingles, en un principio de longitud 2 (dando así $26^2$ posibilidades de shingles), por cada twit se ejecuta la siguiente función de $\verb|minhash|$:

$mh_i(T) = \min_{s \in shingles(T)}\{ h_i(s) \}$

Habrá tantas $mh_i$ y por consiguiente $h_i$ como $r$, en el caso de que se decida hashear el shingle como una cadena se puede utilizar una función de hashing universal para input fijo, por ejemplo
``` python
def fixedlen_hash(a, p, m):
  return lambda x: sum((a[i] * el) % p for i, el in enumerate(x)) % m
```

O bien, identificar cada shingle por un índice y hashearlo con una función de hash más simple (también universal):

``` python
def numeric_hashing(a, b, p, m):
  return lambda x: ((a * x + b) % p) % m
```

Por cada función de minhash y cada tweet se obtiene un valor, formando la tabla:

|  $mh_i$ | T<sub>0</sub> |T<sub>1</sub>|T<sub>3</sub>|
|----|----|---|---|
|$mh_1$|2|1|2|
|$mh_2$|1|2|0|
|$mh_3$|1|1|1|
|$mh_4$|0|1|0|
|$mh_5$|1|1|1|
|$mh_6$|1|1|0|
|$mh_7$|3|3|1|
|$mh_8$|2|3|1|

Tendría que haber $r * b = 4 * 2 = 8$ funciones de minhash(que se cumple), para luego ser mapeada a $b = 4$ bandas.
Una posible distribucíon:

|  $mh_i$ | T<sub>0</sub> |T<sub>1</sub>|T<sub>2</sub>|
|----|----|---|---|
|$mh_1$|2|1|2|
|$mh_2$|1|2|0|
|----|----|---|---|
|$mh_3$|1|1|1|
|$mh_4$|0|1|0|
|----|----|---|---|
|$mh_5$|1|1|1|
|$mh_6$|1|1|0|
|----|----|---|---|
|$mh_7$|3|3|1|
|$mh_8$|2|3|1|


Por cada banda se distribuián los twits:

| banda 1 ||
|----|----|
|0|$\{ T_2 \}$|
|1|$\{ T_0, T_1 \}$|
|2|$\{ T_0, T_1, T_2 \}$|
|3||

<br>

| banda 2 ||
|----|----|
|0|$\{ T_0, T_2 \}$|
|1|$\{ T_0, T_1, T_2 \}$|
|2||
|3||
<br>

| banda 3 ||
|----|----|
|0|$\{ T_0, T_1 \}$|
|1|$\{ T_0, T_1, T_2 \}$|
|2||
|3||

<br>

| banda 4 ||
|----|----|
|0||
|1|$\{ T_2 \}$|
|2|$\{ T_0\}$|
|3|$\{ T_0, T_1 \}$|



#### Ejercicio 3
Describir cómo se hace la predicción de si un tweet es fake news o no utilizando el esquema LSH propuesto. (35 puntos)

----

Dado un twit nuevo, se procede a descomponerlo en shingles tal como se hizo con los primeros, se calculan las distantas funciones de $\verb|minhash|$ (por una cuestión de claridad lo agrego a las tablas anteriores):


|  $mh_i$ | T<sub>0</sub> |T<sub>1</sub>|T<sub>2</sub>|T<sub>3</sub><sup>$Q$</sup>|
|----|----|---|---|---|
|$mh_1$|2|1|2|**1**|
|$mh_2$|1|2|0|**0**|
|----|----|---|---|---|
|$mh_3$|1|1|1|**0**|
|$mh_4$|0|1|0|**1**|
|----|----|---|---|---|
|$mh_5$|1|1|1|**0**|
|$mh_6$|1|1|0|**3**|
|----|----|---|---|---|
|$mh_7$|3|3|1|**2**|
|$mh_8$|2|3|1|**3**|

Ahora por cada franja se procede a recuperar los twees obteniendo así de cada banda:

| banda 1 ||
|----|----|
|0|$\{ T_2 \}$|
|1|$\{ T_0, T_1 \}$|
|2|$\{ T_0, T_1, T_2 \}$|
|3||

$$P_{B_1} = \{T_2\} \cap \{T_0, T_1\} = \{\}$$
<br>

| banda 2 ||
|----|----|
|0|$\{ T_0, T_2 \}$|
|1|$\{ T_0, T_1, T_2 \}$|
|2||
|3||

$$P_{B_2} = \{ T_0, T_2 \} \cap \{ T_0, T_1, T_2 \} = \{ T_0, T_2 \}$$

<br>

| banda 3 ||
|----|----|
|0|$\{ T_0, T_1 \}$|
|1|$\{ T_0, T_1, T_2 \}$|
|2||
|3||

$$P_{B_3} = \{ T_0, T_1 \} \cap \{  \} = \{\}$$

<br>

| banda 4 ||
|----|----|
|0||
|1|$\{ T_2 \}$|
|2|$\{ T_0\}$|
|3|$\{ T_0, T_1 \}$|

$$P_{B_4} = \{ T_0 \} \cap \{ T_0, T_1  \} = \{T_0\}$$


Finalmente ejecutando la etapa $\verb|or|$:

$P_{T_3} = P_{B_1} \cup P_{B_2} \cup P_{B_3} \cup P_{B_4}$

$P_{T_3} = \{\} \cup \{ T_0, T_2 \} \cup \{\} \cup \{T_0\} $

$P_{T_3} = \{ T_0, T_2 \}$

Bastaría ahora hacer:

```python

similars_to_t_3 = {t_0, t_2}
result = []
for tweet in similars_to_t_3:
  if jacard_dist(tweet, t_3) > 0.73:
    result.append(tweet)
```

Si bien, finalmente se realizan comparaciones, no se hacen sobre el set completo, si no, sobre un subconjunto de este, que es deseable que sea lo más chico posible tratando de no introducir muchos falsos negativos.